## Загрузим нужные библиотеки

In [67]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

%matplotlib inline

### Считаем тренировочный набор

In [188]:
df_issues_train = pd.read_csv("./content/train_issues.csv")
df_comment_train = pd.read_csv("./content/train_comments.csv")

df_emp = pd.read_csv("./content/employees.csv")

In [189]:
#emp_positions = df_emp['position'].unique()

## Рассмотрим датасет по ближе

In [190]:
df_issues_train.head(3)

,id,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400


In [191]:
df_issues_train.shape

(9589, 8)

In [192]:
df_issues_train['summary'].value_counts()

Test layout                                        34
Planing task                                       29
Planing tasks                                      24
Work on feedback                                   16
Fix bugs                                           11
                                                   ..
Add correct icons and colors for journeys           1
Testing validation fields in login pages            1
Add loader for "Upload avatar"                      1
Markup improvements for IB curriculums              1
Expand a gitlab parser by adding members emails     1
Name: summary, Length: 9266, dtype: int64

In [193]:
df_issues_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                9589 non-null   int64 
 1   created           9589 non-null   object
 2   key               9589 non-null   object
 3   summary           9589 non-null   object
 4   project_id        9589 non-null   int64 
 5   assignee_id       9589 non-null   int64 
 6   creator_id        9589 non-null   int64 
 7   overall_worklogs  9589 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 599.4+ KB


In [194]:
df_comment_train.head(3)

,comment_id,text,issue_id,author_id
0,11779,[https://www.youtube.com/watch?v=tuhOdtsvoNY|h...,669666,1
1,10601,OK [~accountid:557058:3f7ab89a-8969-4547-90df-...,669670,1
2,76101,I encountered a problem with access to `/users...,670930,2


In [195]:
df_comment_train.shape

(13918, 4)

In [196]:
df_comment_train_sum = df_comment_train.groupby(['issue_id'])['issue_id'].agg([lambda x:x.count()])

In [197]:
df_comment_train_sum = df_comment_train_sum.reset_index()

In [198]:
df_comment_train_sum.columns = ['issue_id', 'comments_count']

In [199]:
df_comment_train_sum

,issue_id,comments_count
0,669666,1
1,669670,1
2,670930,3
3,670934,3
4,675979,5
...,...,...
3940,824411,1
3941,824447,1
3942,923360,1
3943,926256,1


In [200]:
df_comment_train.shape

(13918, 4)

In [201]:
df_comment_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13918 entries, 0 to 13917
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   comment_id  13918 non-null  int64 
 1   text        13918 non-null  object
 2   issue_id    13918 non-null  int64 
 3   author_id   13918 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 435.1+ KB


### Преобразование столбцов

Удалим ненужную информацию о работниках

In [202]:
df_emp = df_emp.drop(["active","full_name", "salary_calculation_type", "passport"],axis = 1)

Обьединим два набора данных по id исполнителя

In [203]:
df_train = pd.merge(df_issues_train, df_emp, left_on="assignee_id", right_on="id", how='inner')

In [204]:
df_train.head()

,id_x,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_y,position,hiring_type,payment_type,english_level,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,Разработчик мобильных приложений,staff,fixed,NaN,1,1,1,1
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,Разработчик мобильных приложений,staff,fixed,NaN,1,1,1,1
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,Разработчик мобильных приложений,staff,fixed,NaN,1,1,1,1
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,Разработчик мобильных приложений,staff,fixed,NaN,1,1,1,1
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,Разработчик мобильных приложений,staff,fixed,NaN,1,1,1,1


In [205]:
df_train.shape

(9589, 17)

Добавим сюда еще информацию о количестве комментариев из третьего набора:

In [206]:
df_train = pd.merge(df_train, df_comment_train_sum, left_on="id_x", right_on="issue_id", how='outer')

In [207]:
df_train.shape

(9589, 19)

In [208]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9589 entries, 0 to 9588
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id_x                        9589 non-null   int64  
 1   created                     9589 non-null   object 
 2   key                         9589 non-null   object 
 3   summary                     9589 non-null   object 
 4   project_id                  9589 non-null   int64  
 5   assignee_id                 9589 non-null   int64  
 6   creator_id                  9589 non-null   int64  
 7   overall_worklogs            9589 non-null   int64  
 8   id_y                        9589 non-null   int64  
 9   position                    7354 non-null   object 
 10  hiring_type                 7630 non-null   object 
 11  payment_type                7529 non-null   object 
 12  english_level               534 non-null    object 
 13  is_nda_signed               9589 

In [209]:
df_train = df_train.fillna(0)

In [210]:
df_train

,id_x,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_y,position,hiring_type,payment_type,english_level,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,issue_id,comments_count
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,Разработчик мобильных приложений,staff,fixed,0,1,1,1,1,0.0,0.0
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,Разработчик мобильных приложений,staff,fixed,0,1,1,1,1,819949.0,1.0
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,Разработчик мобильных приложений,staff,fixed,0,1,1,1,1,0.0,0.0
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,Разработчик мобильных приложений,staff,fixed,0,1,1,1,1,0.0,0.0
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,Разработчик мобильных приложений,staff,fixed,0,1,1,1,1,819941.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688397,2020-01-24 05:40:31.000,XXO-213,Layout - Service and postcode search results p...,33,98,58,18000,98,Web-разработчик,staff,fixed,0,1,1,1,1,688397.0,31.0
9585,670934,2020-06-02 10:08:14.000,LJ-1,Set up the neo4j by using a dataset from Slack,56,2,2,16200,2,Web-разработчик,staff,fixed,0,0,0,0,0,670934.0,3.0
9586,670932,2020-06-05 09:52:12.000,LJ-3,Set up a graph in neo4j by GitLab dataset,56,2,2,3600,2,Web-разработчик,staff,fixed,0,0,0,0,0,0.0,0.0
9587,670931,2020-06-05 13:46:38.000,LJ-4,Parse a dataset from teacherly slack,56,2,2,6300,2,Web-разработчик,staff,fixed,0,0,0,0,0,0.0,0.0


 Создадим список всех не числовых лейблов и взглянем на них

In [211]:
mass_object = df_train.dtypes[df_train.dtypes == "object"].index.values

In [212]:
df_train[mass_object].head(3)

,created,key,summary,position,hiring_type,payment_type,english_level
0,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",Разработчик мобильных приложений,staff,fixed,0
1,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",Разработчик мобильных приложений,staff,fixed,0
2,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",Разработчик мобильных приложений,staff,fixed,0


In [213]:
df_train['summary'].value_counts()

Test layout                                        34
Planing task                                       29
Planing tasks                                      24
Work on feedback                                   16
Fix bugs                                           11
                                                   ..
New view for Subjects and Topics                    1
Templates slider functional                         1
Filter templates by teams                           1
Modal for edit Template parameters                  1
Expand a gitlab parser by adding members emails     1
Name: summary, Length: 9266, dtype: int64

Взглянем на них поближе

In [214]:
df_train['position'].value_counts()

Web-разработчик                             6434
0                                           2235
Разработчик мобильных приложений             305
Системный администратор (DevOps инженер)     250
Менеджер проектов                            168
Директор по IT                                87
Web дизайнер                                  71
DevOps инженер                                29
Инженер тестировщик (QA engineer)              7
Delivery Manager                               2
Директор по стратегическому развитию           1
Name: position, dtype: int64

In [215]:
df_train['position'].unique()

array(['Разработчик мобильных приложений', 'Web-разработчик', 0,
       'Менеджер проектов', 'Системный администратор (DevOps инженер)',
       'Инженер тестировщик (QA engineer)',
       'Директор по стратегическому развитию', 'DevOps инженер',
       'Директор по IT', 'Web дизайнер', 'Delivery Manager'], dtype=object)

In [216]:
df_train['hiring_type'].value_counts()

staff        7005
0            1959
non_staff     625
Name: hiring_type, dtype: int64

In [217]:
df_train['payment_type'].value_counts()

fixed     5765
0         2060
hourly    1764
Name: payment_type, dtype: int64

In [218]:
df_train['english_level'].value_counts()

0     9055
B1     358
C1     176
Name: english_level, dtype: int64

In [219]:
df_train['assignee_id'].value_counts()

1      1864
365    1049
10      801
86      732
33      637
11      498
97      436
68      352
208     329
242     303
192     278
82      250
191     175
330     174
4       172
197     139
291     137
99      129
13      111
384      99
210      87
193      80
94       77
22       73
195      65
53       57
306      50
58       49
147      41
149      38
107      35
93       34
112      29
246      27
57       24
207      21
37       20
54       19
98       13
285      11
264      10
20       10
18       10
8        10
166      10
284       9
206       7
2         4
204       2
116       1
43        1
Name: assignee_id, dtype: int64

Проведем даммизацию:

In [220]:
#df_train_positions = df_train['position'].unique()

In [221]:
#df_train_positions

In [222]:
#emp_positions

In [223]:
#df_add_train_positions = list(set(emp_positions) - set(df_train_positions))

In [224]:
#df_add_train_positions

In [225]:
#df_train = pd.get_dummies(df_train,columns = ['position'], drop_first = True, dummy_na = False)

In [226]:
#df_train.head()

In [227]:
df_train = pd.get_dummies(df_train,columns = ['hiring_type'], drop_first = False, dummy_na = False)

In [228]:
df_train.head()

,id_x,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_y,position,...,english_level,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,issue_id,comments_count,hiring_type_0,hiring_type_non_staff,hiring_type_staff
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,Разработчик мобильных приложений,...,0,1,1,1,1,0.0,0.0,0,0,1
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,Разработчик мобильных приложений,...,0,1,1,1,1,819949.0,1.0,0,0,1
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,Разработчик мобильных приложений,...,0,1,1,1,1,0.0,0.0,0,0,1
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,Разработчик мобильных приложений,...,0,1,1,1,1,0.0,0.0,0,0,1
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,Разработчик мобильных приложений,...,0,1,1,1,1,819941.0,1.0,0,0,1


In [229]:
df_train = pd.get_dummies(df_train,columns = ['payment_type'], drop_first = False, dummy_na = False)

In [230]:
df_train.head()

,id_x,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_y,position,...,is_added_to_internal_chats,is_added_one_to_one,issue_id,comments_count,hiring_type_0,hiring_type_non_staff,hiring_type_staff,payment_type_0,payment_type_fixed,payment_type_hourly
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,Разработчик мобильных приложений,...,1,1,0.0,0.0,0,0,1,0,1,0
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,Разработчик мобильных приложений,...,1,1,819949.0,1.0,0,0,1,0,1,0
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,Разработчик мобильных приложений,...,1,1,0.0,0.0,0,0,1,0,1,0
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,Разработчик мобильных приложений,...,1,1,0.0,0.0,0,0,1,0,1,0
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,Разработчик мобильных приложений,...,1,1,819941.0,1.0,0,0,1,0,1,0


In [231]:
df_train = pd.get_dummies(df_train,columns = ['english_level'], drop_first = False, dummy_na = False)

In [232]:
df_train.head()

,id_x,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_y,position,...,comments_count,hiring_type_0,hiring_type_non_staff,hiring_type_staff,payment_type_0,payment_type_fixed,payment_type_hourly,english_level_0,english_level_B1,english_level_C1
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,Разработчик мобильных приложений,...,0.0,0,0,1,0,1,0,1,0,0
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,Разработчик мобильных приложений,...,1.0,0,0,1,0,1,0,1,0,0
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,Разработчик мобильных приложений,...,0.0,0,0,1,0,1,0,1,0,0
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,Разработчик мобильных приложений,...,0.0,0,0,1,0,1,0,1,0,0
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,Разработчик мобильных приложений,...,1.0,0,0,1,0,1,0,1,0,0


## Выделим выборки

Удалим все лишние столбцы из выборки

In [233]:
#mass_object = df_train.dtypes[df_train.dtypes == "object"].index.values

In [234]:
mass_object = ['created', 'key', 'summary', 'position']

In [235]:
stuff = ["id_x","id_y","overall_worklogs"]
mass_object = np.append(mass_object, stuff)

In [236]:
X = df_train.drop(mass_object, axis = 1)
y = df_train[["overall_worklogs"]]

In [237]:
X.head(3)

,project_id,assignee_id,creator_id,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,issue_id,comments_count,hiring_type_0,hiring_type_non_staff,hiring_type_staff,payment_type_0,payment_type_fixed,payment_type_hourly,english_level_0,english_level_B1,english_level_C1
0,5,93,93,1,1,1,1,0.0,0.0,0,0,1,0,1,0,1,0,0
1,5,93,93,1,1,1,1,819949.0,1.0,0,0,1,0,1,0,1,0,0
2,5,93,93,1,1,1,1,0.0,0.0,0,0,1,0,1,0,1,0,0


In [238]:
X.shape

(9589, 18)

In [239]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Обучение модели

In [247]:
from sklearn.ensemble import RandomForestRegressor

clf = RandomForestRegressor(random_state=0)

In [248]:
clf.fit(X_train, y_train)

C:\Users\Сергей\AppData\Local\Temp\ipykernel_15704\2395142735.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestRegressor(random_state=0)

## Оценка точности

In [249]:
pred = clf.predict(X_test)

In [250]:
from sklearn.metrics import r2_score

score = r2_score(y_test, pred)

In [251]:
score

-0.30475887282630265

## Выдаем предсказания

In [252]:
df_issues_test = pd.read_csv("./content/test_issues.csv")
df_comment_test = pd.read_csv("./content/test_comments.csv")
df_emp = pd.read_csv("./content/employees.csv")

df_emp = df_emp.drop(["active","full_name", "salary_calculation_type", "passport"],axis = 1)

df_test = pd.merge(df_issues_test, df_emp, left_on="assignee_id", right_on="id", how='inner')

df_comment_test_sum = df_comment_test.groupby(['issue_id'])['issue_id'].agg([lambda x:x.count()])
df_comment_test_sum = df_comment_test_sum.reset_index()
df_comment_test_sum.columns = ['issue_id', 'comments_count']

df_test = pd.merge(df_test, df_comment_test_sum, left_on="id_x", right_on="issue_id", how='outer')

df_test = df_test.fillna(0)

df_test = pd.get_dummies(df_test,columns = ['hiring_type'], drop_first = False, dummy_na = False)
df_test = pd.get_dummies(df_test,columns = ['payment_type'], drop_first = False, dummy_na = False)
df_test = pd.get_dummies(df_test,columns = ['english_level'], drop_first = False, dummy_na = False)

df_test = df_test.set_index('id_x')

mass_object = df_test.dtypes[df_test.dtypes == "object"].index.values
stuff = ["id_y"]
mass_object = np.append(mass_object, stuff)

X = df_test.drop(mass_object, axis = 1)
#y = df_test[["overall_worklogs"]]

In [253]:
X

,project_id,assignee_id,creator_id,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,issue_id,comments_count,hiring_type_0,hiring_type_non_staff,hiring_type_staff,payment_type_0,payment_type_fixed,payment_type_hourly,english_level_0,english_level_B1,english_level_C1
id_x,,,,,,,,,,,,,,,,,,
675975,5,112,112,0,0,0,0,675975.0,8.0,0,0,1,0,1,0,1,0,0
675972,5,112,112,0,0,0,0,675972.0,4.0,0,0,1,0,1,0,1,0,0
675917,5,112,1,0,0,0,0,0.0,0.0,0,0,1,0,1,0,1,0,0
675915,5,112,1,0,0,0,0,675915.0,1.0,0,0,1,0,1,0,1,0,0
675912,5,112,1,0,0,0,0,675912.0,1.0,0,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176348,29,191,191,1,1,1,1,2176348.0,1.0,0,0,1,0,0,1,0,0,1
2176673,29,191,192,1,1,1,1,2176673.0,4.0,0,0,1,0,0,1,0,0,1
686533,33,246,58,1,1,1,1,686533.0,2.0,0,0,1,0,0,1,1,0,0


In [254]:
pred = clf.predict(X)

In [255]:
X['overall_worklogs'] = pred

In [256]:
df_submission = X.reset_index()
df_submission = df_submission[['id_x', 'overall_worklogs']]
df_submission

,id_x,overall_worklogs
0,675975,37691.400000
1,675972,26486.400000
2,675917,20458.937372
3,675915,7113.543329
4,675912,7113.543329
...,...,...
1065,2176348,195883.340000
1066,2176673,215086.940000
1067,686533,14623.000000
1068,686395,34513.465061


In [257]:
df_submission.to_csv("./content/RSF_submission1.csv", index=False)